goal_i, goal_j = ij of goal.index(input_v[l = 8]: 6) = 2, 1
input_i, input_j = ij of 8 = 2, 2
man = abs(2+1-2-2) = 1

In [1]:
import random

class PuzzleMaker:
    def __init__(self, n):
        self.n = n
        self.move_names = ['left', 'right', 'top', 'bottom']
        self.proposed_num_moves = random.randint(1, 75)
        self.proposed_move_seq = random.choices(self.move_names, k = self.proposed_num_moves)
        self.mode_names = ['row-asc', 'column-asc', 'row-des', 'column-des']
        n_mode_id = random.randint(0, len(self.mode_names)-1)
        self.chosen_mode = self.mode_names[n_mode_id]
        
        self.true_num_moves = -1
        self.true_move_seq = []
        self.puzzle = []
        
        while self.true_num_moves in [-1, 0]:
            self.make_random_puzzle()
        
    def get_init_puzzle(self):
        puzzle = []
        
        if self.chosen_mode == 'row-asc':
            puzzle = [i+1 for i in range(self.n ** 2)]
            puzzle[-1] = 0
            
        elif self.chosen_mode == 'column-asc':
            puzzle = [0] * (self.n ** 2)
            for i in range(self.n):
                for j in range(self.n):
                    pos = self.get_pos(j ,i)
                    puzzle[pos] = self.get_pos(i, j) + 1
            puzzle[-1] = 0
        
        elif self.chosen_mode == 'row-des':
            puzzle = [i for i in range(self.n ** 2 - 1, -1, -1)]
            
        elif self.chosen_mode == 'column-des':
            puzzle = [0] * (self.n ** 2)
            for i in range(self.n):
                for j in range(self.n):
                    val = self.n**2 - (self.n * i + j + 1)
                    puzzle[self.n * j + i] = val
                            
        return puzzle
        
    def get_ij(self, pos):
        return pos // self.n, pos % self.n
    
    def get_pos(self, i, j):
        return self.n * i + j
    
    def swap_tile(self, curr_state, curr_i, curr_j, target_i, target_j):
        temp_state = curr_state.copy()
        curr_pos = self.get_pos(curr_i, curr_j)
        target_pos = self.get_pos(target_i, target_j)
        tmp_val = temp_state[curr_pos]
        temp_state[curr_pos] = temp_state[target_pos]
        temp_state[target_pos] = tmp_val
        return temp_state
    
    def make_random_puzzle(self):
        puzzle = self.get_init_puzzle()
        zero_i, zero_j = self.get_ij(len(puzzle) - 1)
        
        for step in self.proposed_move_seq:
            if step == 'left':
                if zero_j > 0:
                    left_j = zero_j - 1
                    puzzle = self.swap_tile(puzzle, zero_i, zero_j, zero_i, left_j)
                    zero_j = left_j
                    self.true_move_seq.append('left')
            elif step == 'right':
                if zero_j < self.n - 1:
                    right_j = zero_j + 1
                    puzzle = self.swap_tile(puzzle, zero_i, zero_j, zero_i, right_j)
                    zero_j = right_j
                    self.true_move_seq.append('right')
            elif step == 'top':
                if zero_i > 0:
                    top_i = zero_i - 1
                    puzzle = self.swap_tile(puzzle, zero_i, zero_j, top_i, zero_j)
                    zero_i = top_i
                    self.true_move_seq.append('top')
            elif step == 'bottom':
                if zero_i < self.n - 1:
                    bottom_i = zero_i + 1
                    puzzle = self.swap_tile(puzzle, zero_i, zero_j, bottom_i, zero_j)
                    zero_i = bottom_i
                    self.true_move_seq.append('bottom')
        
        self.true_num_moves = len(self.true_move_seq)
        self.puzzle = puzzle
        
    def get_random_puzzle(self):
        result = {
            'shuffle_num_moves': self.true_num_moves,
            'shuffle_move_seq': self.true_move_seq,
            'puzzle': self.puzzle,
            'mode': self.chosen_mode
        }
        return result

In [2]:
class PuzzleSolver:
    def __init__(self, n, numlist, goal_state):
        self.n = n
        self.numlist = numlist
        self.expanded_states = [] # a unique list
        self.frontier = {} # a dict of actions {tuple(state_n): [g(n), f(n)]}, desc by f(n) 
        self.goal_state = goal_state
        self.search_graph = {} # dict {(next_action): (curr_action, move_name)}
        self.has_solution = True
        self.move_seq = []
        
    def get_zero(self, state):
        zero_id = state.index(0)
        return self.get_ij(zero_id)
    
    def get_ij(self, pos):
        return pos // self.n, pos % self.n
    
    def get_pos(self, i, j):
        return self.n * i + j
    
    def get_indv_manhattan(self, pos, val):
        # print(23)
        goal_pos = self.goal_state.index(val)
        goal_i, goal_j = self.get_ij(goal_pos)
        
        pos_i, pos_j = self.get_ij(pos)
        
        # print(29, val, goal_i, goal_j, pos_i, pos_j)
        
        indv_man_d = abs(goal_i - pos_i) + abs(goal_j - pos_j)
        
        return indv_man_d
    
    def get_hcost_manhattan(self, state):
        # get manhattan distance of all tile 
        # to each's correct position in goal state
        man_d = 0
        for i in range(self.n ** 2):
            man_d += self.get_indv_manhattan(i, state[i])
        # print(39, 'man_d', man_d)
        return man_d
    
    def is_continue_searching(self):
        # state in frontier with least f
        next_state = None
        # print(44)
        if len(self.frontier) > 0:
            # print(45)
            next_in_frontier = list(self.frontier.items())[-1] # tuple (tuple(state), [g, f])
            next_state = list(next_in_frontier[0]) # element at index 0 is the state
            # print('self.frontier', self.frontier)
            # print('next_state', next_state)
            # print('self.goal_state', self.goal_state)
        return next_state != None and next_state != self.goal_state
    
    def swap_tile(self, curr_state, curr_i, curr_j, target_i, target_j):
        temp_state = curr_state.copy()
        curr_pos = self.get_pos(curr_i, curr_j)
        target_pos = self.get_pos(target_i, target_j)
        tmp_val = temp_state[curr_pos]
        temp_state[curr_pos] = temp_state[target_pos]
        temp_state[target_pos] = tmp_val
        return temp_state
    
    def update_search_graph_frontier(self, next_state, curr_state, curr_costs, move_name):
        # print(move_name)
        curr_gcost, curr_fcost = curr_costs
        if next_state not in self.expanded_states:
            next_state_gcost = curr_gcost + 1
            next_state_hcost = self.get_hcost_manhattan(next_state)
            next_state_fcost = next_state_gcost + next_state_hcost
            
            # update search_graph
            next_node = (tuple(next_state), next_state_fcost)
            curr_node = ((tuple(curr_state), curr_fcost), move_name)
            self.search_graph[next_node] = curr_node
            
            # update frontier
            if tuple(next_state) in self.frontier.keys():
                existing_gcost, existing_fcost = self.frontier[tuple(next_state)]
                if existing_fcost > next_state_fcost:
                    # only update when new fcost is less than existing one
                    self.frontier[tuple(next_state)] = [next_state_gcost, next_state_fcost]
            else:
                # new state, just add it in
                self.frontier[tuple(next_state)] = [next_state_gcost, next_state_fcost]
    
    def handle_frontier(self, curr_action):
        curr_state, curr_costs = curr_action
        curr_state = list(curr_state)
        curr_gcost, curr_fcost = curr_costs
        
        zero_i, zero_j = self.get_zero(curr_state)
        
        # top
        if zero_i > 0:
            top_i = zero_i - 1
            top_state = self.swap_tile(curr_state, zero_i, zero_j, top_i, zero_j)
            self.update_search_graph_frontier(top_state, curr_state, curr_costs, 'down')
        
        # bottom
        if zero_i < self.n - 1:
            bottom_i = zero_i + 1
            bottom_state = self.swap_tile(curr_state, zero_i, zero_j, bottom_i, zero_j)
            self.update_search_graph_frontier(bottom_state, curr_state, curr_costs, 'up')
            
        # left
        if zero_j > 0:
            left_j = zero_j - 1
            left_state = self.swap_tile(curr_state, zero_i, zero_j, zero_i, left_j)
            self.update_search_graph_frontier(left_state, curr_state, curr_costs, 'right')
            
        # right
        if zero_j < self.n - 1:
            right_j = zero_j + 1
            right_state = self.swap_tile(curr_state, zero_i, zero_j, zero_i, right_j)
            self.update_search_graph_frontier(right_state, curr_state, curr_costs, 'left')
        
        # descendingly sorting the frontier by fcosts
        self.frontier = dict(sorted(self.frontier.items(), key = lambda item: item[1][1], reverse = True))
    
    def get_move_sequence(self, goal_action, init_node):
        goal_state, goal_costs = goal_action
        goal_gcost, goal_fcost = goal_costs
        goal_node = (goal_state, goal_fcost)
        
#         print('self.search_graph', self.search_graph)
    
        if len(self.search_graph) > 0:
            prev_node, move_name = self.search_graph[goal_node]
            self.move_seq.append((prev_node, move_name))
            while prev_node != init_node:
                prev_node, move_name = self.search_graph[prev_node]
                self.move_seq = [(prev_node, move_name)] + self.move_seq
        
        self.move_seq.append((goal_node, 'Oh no need, the goal is reached now! ^^'))
    
    def solve(self):
        # get zero tile
        zero_i, zero_j = self.get_zero(self.numlist)
                
        # add input state into the frontier with current fcost
        # f(n) = g(n) + h(n)
        # print(137)
        f_init_state = 0 + self.get_hcost_manhattan(self.numlist)
        self.frontier[tuple(self.numlist)] = [0, f_init_state]
        
        # node of init state in search_graph
        init_node = (tuple(self.numlist), f_init_state)
        print('init_node', init_node)
                
        # while there is still states in the frontier 
        # or not reaching goal state yet
        while self.is_continue_searching():
            curr_action = self.frontier.popitem() # move = (tuple(state), [g, f])
            curr_state = curr_action[0]
            
            # handle expanded_states: marked this state as visited
            self.expanded_states.append(curr_state)
            
            # handle frontier: add next possible actions to frontier
            self.handle_frontier(curr_action)
            
        # found goal state is now in frontier with least f or no solution
        found_goal_action = None
        if len(self.frontier) > 0:
            found_goal_action = self.frontier.popitem()
            self.get_move_sequence(found_goal_action, init_node)    
            
        if found_goal_action == None:
            self.has_solution = False
        
        solution = {
            'has_solution': self.has_solution,
            'move_sequence': self.move_seq,
            'num_moves': len(self.move_seq) - 1 # not counting init state
        }
        
        return solution

In [3]:
import multiprocessing as mp
import threading
from time import sleep

class PuzzleSolverManager:
    def __init__(self, n, numlist):
        self.n = n
        self.numlist = numlist
        self.max_solving_time = 600
        self.result_report_filename = 'minimumNBR.txt'
    
    def getNumInversion(self, numlist, order):
        num_inversion = 0
        for i in range(len(numlist) - 1):
            for j in range(1, len(numlist)):
                if numlist[i] > 0 and numlist[j] > 0:
                    if order == 'asc' and numlist[i] > numlist[j]:
                        num_inversion += 1
                    elif order == 'des' and numlist[i] < numlist[j]:
                        num_inversion += 1
        return num_inversion
    
    def get_ij(self, pos):
        return pos // self.n, pos % self.n
    
    def get_pos(self, i, j):
        return self.n * i + j
    
    def get_zero_ij(self, numlist):
        zero_id = numlist.index(0)
        return self.get_ij(zero_id)
    
    def check_solvability(self, numlist, order):
        solvable = False
        num_inversion = self.getNumInversion(numlist, order)
        print('num_inversion',num_inversion)
        if self.n % 2 == 1:
            if num_inversion % 2 == 0:
                solvable = True
        else:
            zero_i, zero_j = self.get_zero_ij(numlist)
            if (self.n - zero_i) % 2 == 0 and num_inversion % 2 == 1:
                solvable = True
            elif (self.n - zero_i) % 2 == 1 and num_inversion % 2 == 0:
                solvable = True
        return solvable
       
    def get_rowswap_numlist(self, numlist):
        rowswap = [0] * len(numlist)
        for i in range(self.n):
            for j in range(self.n):
                rowswap[self.get_pos(i, j)] = numlist[self.get_pos(j, i)]
        return rowswap        
    
    def check_solvability_both_modes(self):
        rowswap_numlist = self.get_rowswap_numlist(self.numlist)
        mode_1_solvable = self.check_solvability(self.numlist, 'asc') # assume mode 1 is row-asc
        mode_2_solvable = self.check_solvability(rowswap_numlist, 'asc') # so mode 2 is col-asc
        mode_3_solvable = self.check_solvability(self.numlist, 'des') # assume mode 3 is row-des
        mode_4_solvable = self.check_solvability(rowswap_numlist, 'des') # so mode 4 is col-des
        all_solvable = mode_1_solvable or mode_2_solvable or mode_3_solvable or mode_4_solvable
        print('INITIAL SOLVABILITY CHECK:', mode_1_solvable, mode_2_solvable, mode_3_solvable, mode_4_solvable)
        return all_solvable 
        
    def get_row_asc_goal(self):
        goal_state_row_asc = [i+1 for i in range(self.n ** 2)]
        goal_state_row_asc[-1] = 0
        return goal_state_row_asc
    
    def get_col_asc_goal(self):
        goal_state_col_asc = [0] * (self.n ** 2)
        for i in range(self.n):
            for j in range(self.n):
                val = self.n * i + j + 1
                if val == self.n ** 2:
                    val = 0
                goal_state_col_asc[self.n * j + i] = val
        return goal_state_col_asc
    
    def get_row_des_goal(self):
        goal_state_row_des = [i for i in range(self.n ** 2 - 1, -1, -1)]
        return goal_state_row_des
    
    def get_col_des_goal(self):
        goal_state_col_des = [0] * (self.n ** 2)
        for i in range(self.n):
            for j in range(self.n):
                val = self.n**2 - (self.n * i + j + 1)
                goal_state_col_des[self.n * j + i] = val
        return goal_state_col_des
    
    def solver_process(self, goal_state, mode, result_holder, i, quit, found):
        print(f'Solve for {mode}')
        result = None
        while not quit.is_set():
            puzzle_solver = PuzzleSolver(self.n, self.numlist, goal_state)
            result = puzzle_solver.solve()
            if result['has_solution'] == True:
                result_holder[i] = tuple((result, mode))
                found.set()
                print(f'Found result for {mode}')
                break
            sleep(0.1)
        print(f'Finish {mode}')
        
    def timer_process(self, quit, found):
        print(f'Start timer for {self.max_solving_time} seconds')
        while not quit.is_set():
            sleep(self.max_solving_time)
            print(f'Solver timer\'s up')
            found.set()
    
    def stringify_solve_process(self, move_seq):
        txt = ''
        for move in move_seq:
            curr_node, next_move = move
            curr_state, fcost = curr_node
            for i in range(self.n):
                for j in range(self.n):
                    val = curr_state[self.get_pos(i, j)]
                    if val == 0:
                        val = '■'
                    txt += f'{val}\t'
                txt += '\n'
            txt += '\n'
            txt += f'Next move: {next_move}'
            txt += '\n'
        return txt
    
    def report_result(self, result_holder):
        report_txt = ''
        has_any_result = False
        for process_result in result_holder:
            if process_result != None:
                result, mode = process_result
                # result: {'has_solution', 'move_sequence', 'num_moves'}
                # mode: row-asc, col-asc, row-des, col-des
                if result['has_solution'] == True:
                    has_any_result = True
                    report_txt += 'A LEAST-MOVE SOLUTION IS FOUND!\n'
                    report_txt += 'ORDER: {}\n'.format(mode)
                    report_txt += 'NUMBER OF MOVES: {}\n'.format(result['num_moves'])
                    report_txt += 'MOVE SEQUENCE:\n'
                    report_txt += self.stringify_solve_process(result['move_sequence'])
        
        if has_any_result == False:
            report_txt += 'NO SOLUTION IS FOUND!'
        
        with open(self.result_report_filename, 'w', encoding='utf-8') as f:
            f.write(report_txt)
        
        print('Wrote report, finished!')
        
            
    def solve_all(self):
        row_asc_goal = self.get_row_asc_goal()
        col_asc_goal = self.get_col_asc_goal()
        row_des_goal = self.get_row_des_goal()
        col_des_goal = self.get_col_des_goal()
        goal_states = [row_asc_goal, col_asc_goal, row_des_goal, col_des_goal]
        modes = ['row-asc', 'col-asc', 'row-des', 'col-des']
        result_holder = [None] * len(goal_states)
        
        if self.check_solvability_both_modes():
            quit_event = mp.Event()
            found_event = mp.Event()

            for i in range(len(goal_states)):
                goal_state = goal_states[i]
                mode = modes[i]
                process = threading.Thread(target = self.solver_process, 
                                     args=(goal_state, mode, result_holder, 
                                           i, quit_event, found_event))
                process.start()

            timer_process = threading.Thread(target = self.timer_process, 
                                             args=(quit_event, found_event))
            timer_process.start()

            found_event.wait()
            quit_event.set()
        
        self.report_result(result_holder)

In [4]:
n = 4

numlist = [6,15,14,3, 2,4,11,8, 12,7,1,13, 10,0,9,5]

# SOLVABLE
# puzzleMaker = PuzzleMaker(n)
# puzzle_data = puzzleMaker.get_random_puzzle()
# print(puzzle_data)
# numlist = puzzle_data['puzzle']


puzzle_solver_manager = PuzzleSolverManager(n, numlist)
puzzle_solver_manager.solve_all()
# goal_state = [1, 2, 3, 4, 12, 13, 14, 5, 11, 0, 15, 6, 10, 9, 8, 7]
# goal_state = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0]
# goal_state = [i+1 for i in range(n**2)]
# goal_state[-1] = 0
# puzzle_solver = PuzzleSolver(n, numlist, goal_state)
# r = puzzle_solver.solve()
# print(r)

num_inversion 92
num_inversion 92
num_inversion 91
num_inversion 91
INITIAL SOLVABILITY CHECK: True True False False
Solve for row-asc
init_node ((6, 15, 14, 3, 2, 4, 11, 8, 12, 7, 1, 13, 10, 0, 9, 5), 42)
Solve for col-asc
init_node ((6, 15, 14, 3, 2, 4, 11, 8, 12, 7, 1, 13, 10, 0, 9, 5), 44)
Solve for row-des
init_node ((6, 15, 14, 3, 2, 4, 11, 8, 12, 7, 1, 13, 10, 0, 9, 5), 38)
Solve for col-des
init_node ((6, 15, 14, 3, 2, 4, 11, 8, 12, 7, 1, 13, 10, 0, 9, 5), 36)
Start timer for 600 seconds
Solver timer's up
Wrote report, finished!


In [5]:
# import random
# import threading
# from time import sleep

# def worker(i, quit, foundit):
#     print ("%d started" % i)
#     while not quit.is_set():
#         x = random.random()
#         if x > 0.7:
#             print ('%d found %g' % (i, x))
#             foundit.set()
#             break
#         sleep(0.1)
#     print ("%d is done" % i)

# if __name__ == "__main__":
#     import multiprocessing as mp
#     quit = mp.Event()
#     foundit = mp.Event()
#     for i in range(mp.cpu_count()):
#         p = threading.Thread(target=worker, args=(i, quit, foundit))
#         p.start()
#         p.join()
#     foundit.wait()
#     quit.set()

In [6]:
# import multiprocessing 

# def do_something():
#     print('Sleeping 1 second...')
#     time.sleep(1)
#     print('Done sleeping')

# p1 = multiprocessing.Process(target=do_something)
# p2 = multiprocessing.Process(target=do_something)

# p1.start()
# p2.start()

In [7]:
# import threading
# import time

# def do_something():
#     print('Sleeping 1 second...')
#     time.sleep(1)
#     print('Done sleeping')

# t1 = threading.Thread(target=do_something)
# t2 = threading.Thread(target=do_something)
# t1.start()
# t2.start()

Solver timer's up
